# Adding the crop mask to OWS (and terria maps)

Once we've indexed the crop-mask into the datacube, we then need to add it to OWS and terria maps so it can be visualised on [maps.digitalearth.africa](https://maps.digitalearth.africa/), and streamed into GIS applications like QGIS and ArcGIS


3. Ensure the `datakube-apps` and `datakube` repositories have correctly configured pod/argo templates. Make sure the image version and config urls are correct.  For production, the files to consider are:

* Batch run job template: [06_stats_crop_mask.yaml](https://bitbucket.org/geoscienceaustralia/datakube-apps/src/master/workspaces/deafrica-prod-af/processing/statistician/06_stats_crop_mask.yaml)

    * Node group configuration (usually you shouldn't need to alter this): [workers.tf](https://bitbucket.org/geoscienceaustralia/datakube/src/37fbf47358d287aecefbe4f079bf5048f0295b82/workspaces/deafrica-prod-af/01_odc_eks/workers.tf#lines-126)  
    
    * Pod template (use this for testing): [06_stats_crop_mask_dev_pod.yaml](https://bitbucket.org/geoscienceaustralia/datakube-apps/src/master/workspaces/deafrica-prod-af/processing/statistician/06_stats_crop_mask_dev_pod.yaml)

4. Use your dev-box to access the `deafrica-prod-af-eks` environment

        setup_aws_vault deafrica-prod-af-eks
        ap deafrica-prod-af-eks

5. Create a dev-pod by running:
            
        kubectl apply -f workspaces/deafrica-prod-af/processing/statistician/06_stats_crop_mask_dev_pod.yaml
    

6. Access the dev-pod by runnning:

        kubectl exec -it crop-mask-dev-pod -n processing -- bash
        

7. Once in the dev-pod we need to create a large set of database 'tasks' that odc-stats will use as inputs to our ML pipeline (tile information). Run:

        odc-stats save-tasks --frequency annual --grid africa-10 gm_s2_semiannual
        

8. The database files created by running the above command now needs to be synced to the product's S3 bucket. Run:
    
        aws s3 sync . s3://deafrica-services/<product>/<version>/db-files --acl bucket-owner-full-control


9. To execute a batch run, we need to publish a list of tiles to AWS's Simple Queue Service. The command `cm-tsk` will use a geojson (e.g. `Western.geojson`) to clip the tasks to just a single region of Africa (defined by the extent of the geojson), and send those tasks/messages to SQS.

        cm-task --task-csv=gm_s2_semiannual_all.csv --geojson=/western/Western.geojson --outfile=/tmp/aez.csv --sqs deafrica-prod-af-eks-stats-crop-mask --db=s3://deafrica-services/crop_mask_eastern/1-0-0/gm_s2_semiannual_all.db
  
  
10. Exit the dev-pod using `exit`, and then trigger the batch run using the command: 

        kubectl -n processing apply -f workspaces/deafrica-prod-af/processing/statistician/06_stats_crop_mask.yaml


10. To monitor the batch run you can use:
   
     - CPU and memory monitoring: https://mgmt.digitalearth.africa/d/wIVvTqR7k/crop-mask
     - SQS and instance monitoring: https://monitor.cloud.ga.gov.au/d/n2TdQCnnz/crop-mask-dev-deafrica?orgId=1

11. To move deadletter items back into the SQS queue, go into the dev pod, start python and run the following:
        
        >>> from odc.aws.queue import redrive_queue
        >>> redrive_queue('deafrica-prod-af-eks-stats-crop-mask-deadletter', 'deafrica-prod-af-eks-stats-crop-mask') -->


## Other useful run notes

* Restarting the job if timeout errors prevent all messages being consumed:

     - Check the logs of multiple jobs
     - If multiple logs show the time-out error, then delete the job with `kubectl -n processing delete jobs crop-mask-ml-job`
     - Restart the job: `kubectl apply -f workspaces/deafrica-prod-af/processing/06_stats_crop_mask.yaml -n processing`

* To delete all messages from SQS queue:

     - go to AWS central app, open SQS, click on the queue you want to remove and hit the delete button

* To list tiles in a s3 bucket; useful to know if results have been successfully written to disk
        
        aws s3 ls s3://deafrica-data-dev-af/crop_mask_western/
        
* To sync (copy) results in a s3 bucket to your local machine
        
        aws s3 sync s3://deafrica-data-dev-af/crop_mask_western/ crop_mask_western

* If doing test runs, and you wish delete test geotifs from the dev bucket
        
        aws s3 rm --recursive s3://deafrica-data-dev-af/crop_mask_western --dryrun

* To test running one or two tiles in the dev-pod, you can directly run the `cm-pred` command


## Step 1: Update the crop-mask OWS config file

* In DE Africa [config github repository](https://github.com/digitalearthafrica/config), navigate to the [ows_crop_mask_cfg.py](https://github.com/digitalearthafrica/config/blob/master/services/ows_refactored/agriculture/ows_crop_mask_cfg.py) file and add the new crop mask layer to the config file, following the conventions of the other crop mask layers (if re-running a crop mask that already exists then this may already exist). e.g this [code block](https://github.com/digitalearthafrica/config/blob/master/services/ows_refactored/agriculture/ows_crop_mask_cfg.py#L205-L245).  For a new crop mask, you'll need to update the `title`, `name`, and `abstract` to reflect the details of the new crop mask (countries it covers, accuracy figures, etc.)


## Step 2: Update the Production and Dev inventory files

* Add the new crop mask layer to both the [dev_af](https://github.com/digitalearthafrica/config/blob/master/services/inventory/dev_af/inventory.json) and [prod_af](https://github.com/digitalearthafrica/config/blob/master/services/inventory/prod_af/inventory.json) inventory jsons.
* In both files you need to iterate the [total_layers_count](https://github.com/digitalearthafrica/config/blob/master/services/inventory/prod_af/inventory.json#L2), and add the new layer in the same style as the other crop-masks e.g. copy and modify this [code block](https://github.com/digitalearthafrica/config/blob/master/services/inventory/prod_af/inventory.json#L352-L363)

## Step 3: Run `explorer-ows-update` on argo.dev

* Once your changes to the config files have been merged through a pull request, we can now update OWS using the Argo worklfow template. Go to a
[argo.dev](https://argo.dev.digitalearth.africa/), hit the `submit new worklfow` button and select the `explorer-ows-update` workflow.  The parameters of the workflow do not need to be changed, simply hit `submit`. This process will take about 30 mins to complete.

<img align="center" src="../figs/ows-update.PNG" width="600">

## Step 4: Inspect the layer in DE Africa maps portal

* At this point, the layer is only available in OWS-dev, meaning it will not automatically show up in the production maps portal at `maps.digitalearth.africa`. However, we can view it in the maps portal by manually adding the dev layers through the `my data` tab in the `explore map data` menu.
* Add the url https://ows.dev.digitalearth.africa to the `add web data` page. Once the dev OWS layers have loaded, navigate to the new crop mask layer e.g. `Agriculture --> Cropland extent map (provisional) --> Cropland extent 2019 Southern Africa` and hit the `+` symbol to add it the maps portal.  We can then inspect and validate that the crop mask is correct and renders correctly.
* If you can't see the layer and you've followed all the right steps, then you may need to manually restart the pod that runs the OWS web service application. See your favourite cloud developer for assistance as this is not yet a self-sevice or automated process.  

<img align="center" src="../figs/terria_dev_ows.PNG" width="700">


## Step 5: Run `explorer-ows-update` on argo production

* If you're happy with the crop mask after exploring it through OWS-dev, its now time to move the crop mask into the production OWS - this will make the crop mask public to everyone on the maps portal.
* Before running the next step, the dataset needs to be indexed into the production database, you may or may not have done this in the `2_indexing_results_into_datcube.ipynb` step. If not, revisit that step and index the dataset using the `s3-indexing` workflow on production Argo.
* Assumng the dataset is in the production database, go to the production Argo webpage: https://argo.digitalearth.africa, and run the same workflow as you did for the argo.dev i.e run the `explorer-ows-update` workflow.

## Step 6: Update terria catalogue

* Next, we need to update the terria catalogue so the new product will be listed on the maps portal
* Add the new crop mask to the [config/terria/africa-prod.json](https://github.com/digitalearthafrica/config/blob/master/terria/africa-prod.json) file following the same pattern as the other crop masks in the catalogue, e.g. this [code block](https://github.com/digitalearthafrica/config/blob/181d3eacd0f7930ac8bcf269b913941f78ae2bd1/terria/africa-prod.json#L597-L613)

## Step 7: Trigger a new tagged release


* The last step is to generate a new tagged release on the `config` repository, this will make the new layer visible on the production DE Africa maps portal.
* On the first page of the `config` repo, hit the `tags` button
* Then, under the `Releases` tab, hit `Draft a new release`
* Choose a tag number that iterates on the previous tags e.g. if the last tag is `0.1.40`, then the new tag should be `0.1.41`
* Add a title and document the release in the boxes provided, then git the green `Pulish Release` button to trigger the new release

<img align="center" src="../figs/config_tags.PNG" width="700">

## You're finished!

The product should now be available to view in the maps portal. If its not there, wait an hour for the ows-update and ECR sync automatic processes to complete (they happen once every hour). If its still not avialable, then again contact your friendly cloud developer and kindly ask they restart the pod running the OWS web application.